In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"shanegerami","key":"0c32ffffd343776b8cd094b718ef52a5"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d gpiosenka/good-guysbad-guys-image-data-set

good-guysbad-guys-image-data-set.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/good-guysbad-guys-image-data-set.zip

Archive:  /content/good-guysbad-guys-image-data-set.zip
replace EfficientNetB3-character-0.98.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import numpy as np
from tqdm import tqdm

In [ ]:
savory_path = '/content/train/savory'
unsavory_path = '/content/train/unsavory'

savory_files = [os.path.join(savory_path, f) for f in os.listdir(savory_path)]
unsavory_files = [os.path.join(unsavory_path, f) for f in os.listdir(unsavory_path)]

all_files = savory_files + unsavory_files
labels = [1] * len(savory_files) + [0] * len(unsavory_files)

In [ ]:
train_files, val_files, train_labels, val_labels = train_test_split(
    all_files, labels, test_size=0.2, random_state=42
)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)

        return img, label

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = CustomDataset(train_files, train_labels, transform=transform)
val_dataset = CustomDataset(val_files, val_labels, transform=transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification

model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)

model.classifier = nn.Linear(in_features=model.config.hidden_size, out_features=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN thi

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=images, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    model.eval()
    all_val_preds = []
    all_val_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(pixel_values=images, labels=labels)
            val_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            all_val_preds.extend(val_preds)
            all_val_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(all_val_labels, all_val_preds)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {val_accuracy}")

Epoch 1/5, Validation Accuracy: 0.9728163992869875
Epoch 2/5, Validation Accuracy: 0.9763814616755794
Epoch 3/5, Validation Accuracy: 0.9786096256684492
Epoch 4/5, Validation Accuracy: 0.9759358288770054
Epoch 5/5, Validation Accuracy: 0.9781639928698752


In [ ]:
google_drive_path = '/content/drive/MyDrive/face_recog_vit.pth'
torch.save(model.state_dict(), google_drive_path)

In [ ]:
#Round 2: data augmentation

In [ ]:
class AugmentedDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)

        return img, label

In [ ]:
# Use the same data augmentation transform as during training
augmentation_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
])

# Create augmented dataset for evaluation
augmented_val_dataset = AugmentedDataset(val_files, val_labels, transform=augmentation_transform)

# Create DataLoader for augmented validation dataset
augmented_val_loader = DataLoader(augmented_val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()

    train_loss = 0.0  # Initialize training loss
    total_train_samples = 0

    # Use tqdm to create a progress bar for training
    with tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as train_bar:
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(pixel_values=images, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            total_train_samples += images.size(0)

            train_bar.set_postfix(train_loss=train_loss / total_train_samples)

    # Calculate average training loss
    avg_train_loss = train_loss / total_train_samples

    model.eval()
    all_val_preds = []
    all_val_labels = []
    val_accuracy = 0.0
    total_val_samples = 0

    # Use tqdm to create a progress bar for validation
    with tqdm(augmented_val_loader, desc=f'Validation', unit='batch') as val_bar:
        with torch.no_grad():
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(pixel_values=images, labels=labels)
                val_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
                all_val_preds.extend(val_preds)

                # Convert labels to a tensor before extending the list
                all_val_labels.extend(labels.cpu().numpy())

                total_val_samples += images.size(0)
                val_accuracy = accuracy_score(all_val_labels, all_val_preds)

                val_bar.set_postfix(val_accuracy=val_accuracy)

    print(f"Epoch {epoch + 1}/{num_epochs}, Avg Training Loss: {avg_train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Validation: 100%|██████████| 36/36 [00:18<00:00,  1.98batch/s, val_accuracy=0.782]


Epoch 1/5, Avg Training Loss: 0.1203, Validation Accuracy: 0.7816


Validation: 100%|██████████| 36/36 [00:17<00:00,  2.01batch/s, val_accuracy=0.749]


Epoch 2/5, Avg Training Loss: 0.0473, Validation Accuracy: 0.7491


Validation: 100%|██████████| 36/36 [00:17<00:00,  2.02batch/s, val_accuracy=0.723]


Epoch 3/5, Avg Training Loss: 0.0333, Validation Accuracy: 0.7233


Validation: 100%|██████████| 36/36 [00:17<00:00,  2.04batch/s, val_accuracy=0.725]


Epoch 4/5, Avg Training Loss: 0.0178, Validation Accuracy: 0.7255


Validation: 100%|██████████| 36/36 [00:17<00:00,  2.05batch/s, val_accuracy=0.713]

Epoch 5/5, Avg Training Loss: 0.0207, Validation Accuracy: 0.7130


In [ ]:
google_drive_path = '/content/drive/MyDrive/face_recog_vit_aug1.pth'
torch.save(model.state_dict(), google_drive_path)